In [1]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('week5GraphTask2').enableHiveSupport().master('yarn').getOrCreate()

In [7]:
spark

In [58]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number,col,sum,count

In [11]:
data = spark.read.parquet("/data/sample264")

In [12]:
# data.show()

+------+-------+--------+----------+
|userId|trackId|artistId| timestamp|
+------+-------+--------+----------+
| 13065| 944906|  978428|1501588527|
|101897| 799685|  989262|1501555608|
|215049| 871513|  988199|1501604269|
|309769| 857670|  987809|1501540265|
|397833| 903510|  994595|1501597615|
|501769| 818149|  994975|1501577955|
|601353| 958990|  973098|1501602467|
|710921| 916226|  972031|1501611582|
|  6743| 801006|  994339|1501584964|
|152407| 913509|  994334|1501571055|
|256855| 922936| 1001905|1501592059|
|366423| 858904|  974503|1501571685|
|479319| 816927|  975237|1501589184|
|589911| 960078|  995315|1501555145|
|673367| 825174|  970511|1501549783|
|  2237| 933052| 1000942|1501574276|
|108477| 850266|  974937|1501570638|
|225725| 878963|  986907|1501574389|
|302781| 934017|  974572|1501600590|
|399037| 901023|  974580|1501603463|
+------+-------+--------+----------+
only showing top 20 rows



In [43]:
datagrp = data.groupBy(col('userId'),col('trackId')).agg(count(col('userId')).alias('ct'))

In [49]:
window = Window.partitionBy(col('userId')).orderBy(col('ct').desc())

In [57]:
# datagrp.filter(col('userId') == 13065).show()

+------+-------+---+
|userId|trackId| ct|
+------+-------+---+
| 13065| 922861|  1|
| 13065| 937443|  1|
| 13065| 802612|  1|
| 13065| 838399|  1|
| 13065| 863025|  1|
| 13065| 865663|  1|
| 13065| 829407|  1|
| 13065| 868885|  1|
| 13065| 963199|  2|
| 13065| 842664|  1|
| 13065| 848614|  1|
| 13065| 949510|  1|
| 13065| 944906|  1|
| 13065| 844957|  1|
+------+-------+---+



In [61]:
datalmt = datagrp.withColumn('row_number',row_number().over(window))\
    .filter(col('row_number')<= 1000).drop(col('row_number'))

In [65]:
# datalmt.show()

+------+-------+---+
|userId|trackId| ct|
+------+-------+---+
|  3175| 947718|  1|
|  3175| 940951|  1|
|  3175| 845631|  1|
|  3175| 864690|  1|
|  3175| 831005|  1|
|  3175| 930432|  1|
|  3175| 965012|  1|
|  3175| 858940|  1|
|  3175| 829307|  1|
|  5518| 961148|  3|
|  5518| 873588|  2|
|  5518| 930964|  1|
|  5803| 810419|  1|
|  6654| 802183|  1|
|  6654| 886091|  1|
|  6654| 825094|  1|
|  6654| 846873|  1|
|  6654| 928999|  1|
|  7253| 800755|  1|
|  7253| 935631|  1|
+------+-------+---+
only showing top 20 rows



In [77]:
datalmtgrp = datalmt.groupBy(col('userId')).agg(sum(col('ct')).alias('sum_ct'))

In [79]:
data_norm  = datagrp.join(datalmtgrp,'userId','inner').withColumn('norm',col('ct')/col('sum_ct'))

In [85]:
res = data_norm.orderBy(col('norm').desc(),col('userId'),col('trackId')).limit(40)

In [86]:
res_final = res.select(col('userId'),col('trackId'))

In [87]:
for t1,t2 in res_final.collect():
    print('{}\t{}'.format(t1,t2))

66	965774
116	867268
128	852564
131	880170
195	946408
215	860111
235	897176
300	857973
321	915545
328	943482
333	818202
346	864911
356	961308
428	943572
431	902497
445	831381
488	841340
542	815388
617	946395
649	901672
658	937522
662	881433
698	935934
708	952432
746	879259
747	879259
776	946408
784	806468
806	866581
811	948017
837	799685
901	871513
923	879322
934	940714
957	945183
989	878364
999	967768
1006	962774
1049	849484
1057	920458
